# LangChain test

In [1]:
import os
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

/home/julio/Dev/SCG_IFSC/aisurveywriter/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_core.messages import HumanMessage

llm.invoke([HumanMessage(content="Write a poem about the moon")])

In [7]:
# Get pdf data from references
from langchain.document_loaders import PyPDFLoader

pdf_files = [
    "../refexamples/ArigaK2023_Langmuir.pdf",
    "../refexamples/FangC_ApplicationsLangmuir.pdf",
]
doc_data = []
for file in pdf_files:
    loader = PyPDFLoader(file)
    doc_data.extend(loader.load())

In [8]:
import yaml
from langchain.prompts import PromptTemplate

with open("../paper_instructions.yaml", "r") as f:
    paper = yaml.safe_load(f)

pdf_data = "\n".join([doc.page_content for doc in doc_data])
prompt_fmt = paper["base_prompt_format"] + "\n\nThe accompanying PDF data for the references is:\n{pdf_data}"
prep_instructions = paper["preparation_instructions"]
prompt = PromptTemplate(
    input_variables=["preparation_instructions", "title", "subject", "description"],
    template=paper["base_prompt_format"],
)

In [10]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
gen_sections = []

for section in paper["sections"]:
    response = chain.run({
        "preparation_instructions": prep_instructions,
        "subject": paper["subject"],
        "pdf_data": pdf_data,
        "title": section["title"],
        "description": section["description"],
    })
    gen_sections.append({"title": section["title"], "content": response})

In [11]:
print(gen_sections)

[{'title': 'Introduction', 'content': '\\documentclass{article}\n\\usepackage{tikz}\n\\usepackage{graphicx}\n\n\\begin{document}\n\n\\section{Introduction}\n\n\\subsection{Definition and Importance of Langmuir Monolayers and Langmuir-Blodgett Films}\n\nLangmuir monolayers (LMs) and Langmuir-Blodgett (LB) films are ultrathin, organized molecular assemblies formed at the air-water interface and subsequently transferred onto solid substrates (KUMARESAN et al., 2022).  LMs are essentially a single layer of amphiphilic molecules, possessing both hydrophilic (water-loving) and hydrophobic (water-repelling) portions, spontaneously self-assembling at the air-water interface to minimize their free energy (PETTY, 1996).  The hydrophobic tails are oriented away from the water, while the hydrophilic heads are immersed in the water subphase.  By carefully compressing this monolayer using a movable barrier, a highly ordered, two-dimensional structure can be achieved, allowing for precise control ove

In [13]:
dump = {"sections": gen_sections}
with open("generated20241213", "w") as f:
    yaml.dump(dump, f)